## Adult Income Dataset

## Import Requisite Libraries

In [1]:
from ucimlrepo import fetch_ucirepo
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from model_tuner import Model

In [2]:
# fetch dataset
adult = fetch_ucirepo(id=2)

# data (as pandas dataframes)
X = adult.data.features
y = adult.data.targets

In [3]:
print("-" * 80)
print("X")
print("-" * 80)

print(X.head())  # inspect first 5 rows of X

--------------------------------------------------------------------------------
X
--------------------------------------------------------------------------------
   age         workclass  fnlwgt  education  education-num  \
0   39         State-gov   77516  Bachelors             13   
1   50  Self-emp-not-inc   83311  Bachelors             13   
2   38           Private  215646    HS-grad              9   
3   53           Private  234721       11th              7   
4   28           Private  338409  Bachelors             13   

       marital-status         occupation   relationship   race     sex  \
0       Never-married       Adm-clerical  Not-in-family  White    Male   
1  Married-civ-spouse    Exec-managerial        Husband  White    Male   
2            Divorced  Handlers-cleaners  Not-in-family  White    Male   
3  Married-civ-spouse  Handlers-cleaners        Husband  Black    Male   
4  Married-civ-spouse     Prof-specialty           Wife  Black  Female   

   capital-gain  c

In [4]:
print("-" * 80)
print("y")
print("-" * 80)

print(y.head())  # inspect first 5 rows of y


y.loc[:, 'income'] = y['income'].str.rstrip('.')  # Remove trailing periods

# Check the updated value counts
print(y['income'].value_counts())



--------------------------------------------------------------------------------
y
--------------------------------------------------------------------------------
  income
0  <=50K
1  <=50K
2  <=50K
3  <=50K
4  <=50K
income
<=50K    37155
>50K     11687
Name: count, dtype: int64


In [5]:
y.value_counts()

income
<=50K     37155
>50K      11687
Name: count, dtype: int64

In [6]:
y = y['income'].map({'<=50K': 0, '>50K': 1})

In [7]:
outcome = [
    "y"
]

In [8]:
# >2 categories
categorical_features = [
    "workclass",
    "education",
    "marital-status",
    "occupation",
    "relationship",
    "race",
    "sex",
    "native-country",
    "race",
]


# continuous or binary
numerical_features = X.select_dtypes(np.number).columns.to_list()

In [9]:
xgb_name = "xgb"
xgb = XGBClassifier(
    objective="binary:logistic",
    random_state=222,
)
xgbearly = True
tuned_parameters_xgb = {
    f"{xgb_name}__max_depth": [3, 10, 20, 200, 500],
    f"{xgb_name}__learning_rate": [1e-4],
    f"{xgb_name}__n_estimators": [1000],
    f"{xgb_name}__early_stopping_rounds": [100],
    f"{xgb_name}__verbose": [0],
    f"{xgb_name}__eval_metric": ["logloss"],
}

xgb_definition = {
    "clc": xgb,
    "estimator_name": xgb_name,
    "tuned_parameters": tuned_parameters_xgb,
    "randomized_grid": False,
    "n_iter": 5,
    "early": xgbearly,
}

model_definitions = {
    xgb_name: xgb_definition,
}

In [10]:
# Define transformers for different column types
numerical_transformer = Pipeline(
    steps=[
        ("scaler", StandardScaler()),
        ("imputer", SimpleImputer(strategy="mean")),
    ]
)

categorical_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="constant", fill_value="missing")),
        ("encoder", OneHotEncoder(handle_unknown="ignore")),
    ]
)

# Create the ColumnTransformer with passthrough
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numerical_transformer, numerical_features),
        ("cat", categorical_transformer, categorical_features),
    ],
    remainder="passthrough",
)

In [11]:
model_type = "xgb"
clc = xgb_definition["clc"]
estimator_name = xgb_definition["estimator_name"]

tuned_parameters = xgb_definition["tuned_parameters"]
n_iter = xgb_definition["n_iter"]
rand_grid = xgb_definition["randomized_grid"]
early_stop = xgb_definition["early"]
kfold = False
calibrate = True

In [12]:
model_xgb = Model(
    name=f"AIDS_Clinical_{model_type}",
    estimator_name=estimator_name,
    calibrate=calibrate,
    estimator=clc,
    model_type="classification",
    kfold=kfold,
    pipeline_steps=[("ColumnTransformer", preprocessor)],
    stratify_y=True,
    stratify_cols=["race", "sex"],
    grid=tuned_parameters,
    randomized_grid=rand_grid,
    boost_early=early_stop,
    scoring=["roc_auc"],
    random_state=222,
    n_jobs=2,
)

In [13]:
model_xgb.grid_search_param_tuning(X, y, f1_beta_tune=True)

X_train, y_train = model_xgb.get_train_data(X, y)
X_test, y_test = model_xgb.get_test_data(X, y)
X_valid, y_valid = model_xgb.get_valid_data(X, y)


Pipeline Steps:

┌───────────────────────────────────────────────────────────┐
│ Step 1: preprocess_column_transformer_ColumnTransformer   │
│ ColumnTransformer                                         │
└───────────────────────────────────────────────────────────┘
                             │
                             ▼
┌───────────────────────────────────────────────────────────┐
│ Step 2: xgb                                               │
│ XGBClassifier                                             │
└───────────────────────────────────────────────────────────┘



100%|██████████| 5/5 [04:11<00:00, 50.36s/it]


Fitting model with best params and tuning for best threshold ...


100%|██████████| 2/2 [00:01<00:00,  1.75it/s]

Best score/param set found on validation set:
{'params': {'xgb__early_stopping_rounds': 100,
            'xgb__eval_metric': 'logloss',
            'xgb__learning_rate': 0.0001,
            'xgb__max_depth': 10,
            'xgb__n_estimators': 999},
 'score': 0.9054592506968374}
Best roc_auc: 0.905 



In [14]:
model_xgb.fit(
    X_train,
    y_train,
    validation_data=[X_valid, y_valid],
    score="roc_auc",
)

In [15]:
model_xgb.predict(X_test)

array([0, 0, 0, ..., 0, 0, 0])

In [16]:
X_test.shape

(9769, 14)